In [11]:
import cv2
from ultralytics import YOLO
import numpy as np
import joblib
from datetime import datetime
import os
model = YOLO("yolov8n-pose.pt")
knn_model = joblib.load('knn_model.pkl')
knn_scaler = joblib.load('knn_scaler.pkl')
path = "./fall2.mp4"
cap = cv2.VideoCapture(path)

# 將信心值加入關鍵點座標
def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.boxes)):
        box_results = []
        for  j  in range(17):
            x, y = r.keypoints.xyn[i][j].numpy()
            box_results.extend([x, y])
            # confidence = r.keypoints.conf[i][j].item()  # Convert tensor to float
        combined_results.append(box_results)
    return combined_results

# 開始影片偵測
while True:
    ret, frame = cap.read()
    if not ret:
        break
    current_time = datetime.now()
    # Perform pose detection
    results = model(frame, conf = 0.3) #偵測影片中人體關節點.track 可以將偵測到的人加上id
    annotated_frame = results[0].plot() #.plot 將關鍵點畫到影片上
    r = results[0] # r = 輸出結果 
    # #印出 所有關鍵點的座標位置
    # print(r.keypoints.xyn.numpy().reshape(-1, 2)) 
    # #印出 所有關鍵點的信心值
    # print(r.keypoints.conf.numpy()) 
    #將所有關鍵點的位置與信心值丟入knn/svm/random forest/mlp 模型進行預測(0:沒跌倒/1:失衡/2:跌倒)
    if len(results[0].boxes) >= 1:
        data = np.array(process_image(results)) #  將資料轉成2d numpy array
        data = knn_scaler.transform(data)
        y_pred = knn_model.predict(data)
        print(y_pred)
        if y_pred[0] == 2:
            fall_current_time = current_time
            folder_name = 'Fall_img'
            os.makedirs(folder_name, exist_ok=True)
            time_text = fall_current_time.strftime("%Y-%m-%d %H:%M:%S")
            cv2.imwrite(os.path.join(folder_name, fall_current_time.strftime("%Y-%m-%d_%H%M%S_fall.jpg")), frame)
            print("output save")
    # 若是跌倒輸出圖片,並註記時間

    # 存進database

    #通知line，並將圖片送出
    cv2.imshow('YOLOv8 Pose Detection', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 205.6ms
Speed: 2.0ms preprocess, 205.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 86.8ms
Speed: 1.0ms preprocess, 86.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 83.9ms
Speed: 2.0ms preprocess, 83.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 88.0ms
Speed: 1.0ms preprocess, 88.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 89.0ms
Speed: 1.0ms preprocess, 89.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 86.0ms
Speed: 1.0ms preprocess, 86.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 84.9ms
Speed: 1.0ms preprocess, 84.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 86.9ms
Speed: 1.0ms preprocess, 86.9ms

In [ ]:
import csv
from ultralytics import YOLO
# Load YOLO model
image_path = "C:/Users/User/Desktop/Fall Detection/detect/detected_9281.jpg"
model = YOLO("yolov8n-pose.pt")
results = model(image_path)
r = results[0] 
# by 照片人數去抓每個人的keypoints 
# 照片人數 = len(results[0].boxes)
for i in range(len(results[0].boxes)):
    # print(r.boxes.xywhn[i])
    print(r.keypoints.conf[i]) # keypoint共有17個點,每個點yolo偵測到會給一個信心值
    print(r.keypoints.xyn[i]) # keypoint共有17個點,yolo偵測到的x,y座標並做標準化
    print("-----------------")

# Initialize a list to store the combined (x, y, confidence) tuples
combined_results = []

# Iterate over each keypoint
for i in range(len(r.keypoints.xyn)):
    for  j  in range(17):
        x, y = r.keypoints.xyn[i][j].numpy() # Convert tensor to numpy
        confidence = r.keypoints.conf[i][j].item()  # Convert tensor to float
        combined_results.append((x, y, confidence))
print(combined_results)

In [ ]:
import os
import csv
from ultralytics import YOLO
from PIL import Image
# 圖片偵測存檔範例
save_dir = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"
os.makedirs(save_dir, exist_ok=True)



for i in range(1, 209):
    image_path = f"C:/Users/leo86/Desktop/fall_dataset/images/train/fall{i:03d}.jpg"
    results = model.track(image_path, )
    r = results[0]
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    save_path = os.path.join(save_dir, f"fall{i:03d}.jpg")
    im.save(save_path)

In [ ]:
import os
import csv
from ultralytics import YOLO
from PIL import Image
# 圖片偵測存檔範例
save_dir = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"
os.makedirs(save_dir, exist_ok=True)



for i in range(1, 167):
    image_path = f"C:/Users/leo86/Desktop/fall_dataset/images/train/not fallen{i:03d}.jpg"
    results = model.track(image_path, )
    r = results[0]
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    save_path = os.path.join(save_dir, f"not fallen{i:03d}.jpg")
    im.save(save_path)

In [ ]:
import os

directory = "C:/Users/leo86/Desktop/fall_dataset/processed_images/"

missing_files = []

for i in range(1, 209):
    filename = os.path.join(directory, f"fall{i:03d}.jpg")
    if not os.path.exists(filename):
        missing_files.append(i)

if missing_files:
    print("Missing files:")
    print(missing_files)
else:
    print("All files exist.")
